# import Librarys

In [4]:
import pandas as pd
import numpy as np

In [5]:
# create a sample messy dataset
data = {
    'Name': ['John','Alice', 'Bob', 'Emma', np.nan],
    'Age' : [20, 999, 30, -5, 28],
    'Salary': [50000, 60000, np.nan, 45000, 55000],
    'Date': ['2023-01-01', '2023/02/30', '2023-03-15', '2023-04-01', '2023-05-01'],
    'Department': ['IT', 'HR', 'IT', 'Sales', 'HR']
}

In [14]:
df = pd.DataFrame(data)
print(f'Original messy data: ')
print(df)

Original messy data: 
    Name  Age   Salary        Date Department
0   John   20  50000.0  2023-01-01         IT
1  Alice  999  60000.0  2023/02/30         HR
2    Bob   30      NaN  2023-03-15         IT
3   Emma   -5  45000.0  2023-04-01      Sales
4    NaN   28  55000.0  2023-05-01         HR


# step 1 - Handling missing values

In [8]:
# Identify missing values
print("\nMissing values count:")
print(df.isnull().sum())


Missing values count:
Name          1
Age           0
Salary        1
Date          0
Department    0
dtype: int64


In [12]:
# Strategies for handling missing values:

# a) Drop rows with missing values
df_dropped = df.dropna()
print("\nAfter dropping NA:")
print(df_dropped)

# b) Fill with a specific value
df_filled = df.copy()
df_filled['Name'] = df_filled['Name'].fillna('unknown')
df_filled['Salary'] = df_filled['Salary'].fillna(df_filled['Salary'].mean())
print("\nAfter filling NA:")
print(df_filled)

# c) Forward fill or backward fill
df_ffill = df.copy()
df_ffill = df_ffill.ffill()
print("\nAfter forward fill:")
print(df_ffill)


After dropping NA:
    Name  Age   Salary        Date Department
0   John   20  50000.0  2023-01-01         IT
1  Alice  999  60000.0  2023/02/30         HR
3   Emma   -5  45000.0  2023-04-01      Sales

After filling NA:
      Name  Age   Salary        Date Department
0     John   20  50000.0  2023-01-01         IT
1    Alice  999  60000.0  2023/02/30         HR
2      Bob   30  52500.0  2023-03-15         IT
3     Emma   -5  45000.0  2023-04-01      Sales
4  unknown   28  55000.0  2023-05-01         HR

After forward fill:
    Name  Age   Salary        Date Department
0   John   20  50000.0  2023-01-01         IT
1  Alice  999  60000.0  2023/02/30         HR
2    Bob   30  60000.0  2023-03-15         IT
3   Emma   -5  45000.0  2023-04-01      Sales
4   Emma   28  55000.0  2023-05-01         HR


# step 2 - Handling Outliners

In [21]:
# Detect outliers using IQR method
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)][column]
    return outliers

# Check for outliers in Age and Salary
print("\nAge outliers:")
print(detect_outliers(df, 'Age'))
print("\nSalary outliers:")
print(detect_outliers(df, 'Salary'))

# Handle outliers
df_clean = df.copy()
# Cap extreme values
df_clean['Age'] = df_clean['Age'].clip(lower=0, upper=100)
df_clean['Salary'] = df_clean['Salary'].clip(lower=0, upper=100000)
print("\nAfter handling outliers:")
print(df_clean)



Age outliers:
1    999
3     -5
Name: Age, dtype: int64

Salary outliers:
Series([], Name: Salary, dtype: float64)

After handling outliers:
    Name  Age   Salary        Date Department
0   John   20  50000.0  2023-01-01         IT
1  Alice  100  60000.0  2023/02/30         HR
2    Bob   30      NaN  2023-03-15         IT
3   Emma    0  45000.0  2023-04-01      Sales
4    NaN   28  55000.0  2023-05-01         HR


# step 3 - Data transformation

In [22]:
# Convert Date column to datetime
df_clean['Date'] = pd.to_datetime(df_clean['Date'], errors='coerce')
print("\nAfter date conversion:")
print(df_clean)

# Standardize numerical columns
def standardize_column(series):
    return (series - series.mean()) / series.std()

df_clean['Salary_standardized'] = standardize_column(df_clean['Salary'])
print("\nAfter standardization:")
print(df_clean)

# Normalize numerical columns (0 to 1 scale)
def normalize_column(series):
    return (series - series.min()) / (series.max() - series.min())

df_clean['Salary_normalized'] = normalize_column(df_clean['Salary'])
print("\nAfter normalization:")
print(df_clean)

# Encode categorical variables
# One-hot encoding
df_encoded = pd.get_dummies(df_clean, columns=['Department'], prefix='Dept')
print("\nAfter one-hot encoding:")
print(df_encoded)

# Label encoding alternative
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_clean['Department_encoded'] = le.fit_transform(df_clean['Department'].astype(str))
print("\nAfter label encoding:")
print(df_clean)


After date conversion:
    Name  Age   Salary       Date Department
0   John   20  50000.0 2023-01-01         IT
1  Alice  100  60000.0        NaT         HR
2    Bob   30      NaN 2023-03-15         IT
3   Emma    0  45000.0 2023-04-01      Sales
4    NaN   28  55000.0 2023-05-01         HR

After standardization:
    Name  Age   Salary       Date Department  Salary_standardized
0   John   20  50000.0 2023-01-01         IT            -0.387298
1  Alice  100  60000.0        NaT         HR             1.161895
2    Bob   30      NaN 2023-03-15         IT                  NaN
3   Emma    0  45000.0 2023-04-01      Sales            -1.161895
4    NaN   28  55000.0 2023-05-01         HR             0.387298

After normalization:
    Name  Age   Salary       Date Department  Salary_standardized  \
0   John   20  50000.0 2023-01-01         IT            -0.387298   
1  Alice  100  60000.0        NaT         HR             1.161895   
2    Bob   30      NaN 2023-03-15         IT             

# Putting It All Together

In [23]:
def clean_dataset(df):
    # Create a copy to avoid modifying original
    df_cleaned = df.copy()
    
    # Handle missing values
    df_cleaned['Name'] = df_cleaned['Name'].fillna('Unknown')
    df_cleaned['Salary'] = df_cleaned['Salary'].fillna(df_cleaned['Salary'].median())
    
    # Handle outliers
    df_cleaned['Age'] = df_cleaned['Age'].clip(lower=0, upper=100)
    df_cleaned['Salary'] = df_cleaned['Salary'].clip(lower=0, upper=100000)
    
    # Transform data
    df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date'], errors='coerce')
    df_cleaned['Salary_standardized'] = standardize_column(df_cleaned['Salary'])
    
    # Encode categorical variables
    df_cleaned = pd.get_dummies(df_cleaned, columns=['Department'], prefix='Dept')
    
    return df_cleaned

# Apply cleaning function
cleaned_df = clean_dataset(df)
print("\nFinal cleaned dataset:")
print(cleaned_df)


Final cleaned dataset:
      Name  Age   Salary       Date  Salary_standardized  Dept_HR  Dept_IT  \
0     John   20  50000.0 2023-01-01            -0.447214    False     True   
1    Alice  100  60000.0        NaT             1.341641     True    False   
2      Bob   30  52500.0 2023-03-15             0.000000    False     True   
3     Emma    0  45000.0 2023-04-01            -1.341641    False    False   
4  Unknown   28  55000.0 2023-05-01             0.447214     True    False   

   Dept_Sales  
0       False  
1       False  
2       False  
3        True  
4       False  


In [24]:
# Check data types
print(df.dtypes)

# Convert data types if needed
df['Age'] = df['Age'].astype(float)

Name           object
Age             int64
Salary        float64
Date           object
Department     object
dtype: object


In [26]:
# Remove whitespace and standardize string format
df['Name'] = df['Name'].str.strip().str.title()

In [27]:
# Check and remove duplicates
print(f"Number of duplicates: {df.duplicated().sum()}")
df_no_duplicates = df.drop_duplicates()

Number of duplicates: 0
